In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN2001_paper"

# Neural wavelet network
6levels wavelet then 5 layers CNN [2, 2, 2, 2, 2] 

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10


wavelet_levels = 6    #number of wavelet layer to have in the network
convolution_levels = 5    #number of convolution to have in the network
pooling_stride = [2, 2, 2, 2, 2]    #stride of pooling in convolution
weights_shape = [[5, n_variables, 32],
                 [5, 32, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64]]
bias_shape = [32, 64, 64, 64, 64]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
granularity = np.prod(pooling_stride)
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

variables_weights = np.ones((n_classes,))

In [7]:
print pos_weight

3.18601822124


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [10]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [11]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation,
                                 trainable_wavelet = True,
                                 trainable_conv = True,
                                 trainable_last = True,
                                 variables_weights = variables_weights)                        

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    lobal_step = 0,
                    eval_type = 'change')

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100,
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.207057, fmeasure_max= 0.390464, precision= 0.242691, recall= 0.998353
Train Random guess:  , auc= 0.237187fmeasure_max= 0.385662, precision= 0.241480, recall= 0.957166
Validation Iter 0, auc= 0.215012, fmeasure_max= 0.380034, precision= 0.237726, recall= 0.946827
Validation Random guess:  auc= 0.212200, fmeasure_max= 0.371456, precision= 0.228091, recall= 1.000000
Minibatch Loss= 1.074428
Tarin Iter 100, auc= 0.672506, fmeasure_max= 0.653816, precision= 0.639517, recall= 0.668770
Train Random guess:  , auc= 0.242043fmeasure_max= 0.397587, precision= 0.248907, recall= 0.987382
Validation Iter 100, auc= 0.624639, fmeasure_max= 0.622356, precision= 0.556005, recall= 0.706690
Validation Random guess:  auc= 0.230992, fmeasure_max= 0.370983, precision= 0.227734, recall= 1.000000
Minibatch Loss= 0.818587
Tarin Iter 200, auc= 0.676632, fmeasure_max= 0.671454, precision= 0.726214, recall= 0.624374
Train Random guess:  , auc= 0.228630fmeasure_max= 0.381164, precision= 0.2354

In [14]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/DWN2001/DWN2001-10000'

In [15]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.926441 F1_max = 0.846038 prec = 0.840955 rec = 0.851182
Test Random guess: auc= 0.232275 fmeasure_max= 0.376037 , precision= 0.231619 , recall= 0.998818


In [16]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [17]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':probability_change})